## Old Functions and Imports

In [ ]:
%load_ext lab_black

import h5py
import os

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm

from typing import Dict, List, Optional, Tuple
import numpy as np
import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt


import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    cnn,
)

colors = pc.qualitative.D3
colors_seq = pc.sequential.dense
angstrom = "\u212B"
Theta = "\u0398"

In [ ]:
def average_timescans(files, ddir, new_filename):
    datas = []
    for i in range(0, len(files)):
        ad = ARPES_DATA[files[i]]
        datas.append(ad.data)
    data_avg = np.mean(datas, axis=0)
    print(data_avg.shape)

    new_data = data_avg

    new_fn = os.path.join(ddir, new_filename)

    with h5py.File(
        new_fn, "w"
    ) as f:  # Note: 'w' creates a new empty file (or overwrites), use 'r+' to modify an existing file
        f["data"] = new_data.T
        axes_names = [
            "angles",
            "energies",
        ]  # Change these to match your axes labels
        axes = [ad.theta, ad.energy]
        for axis, name in zip(axes, axes_names):
            f[name] = np.atleast_2d(axis).T
        entry_group = f.require_group("entry1")
        entry_group["ScanValues"] = np.atleast_2d(ad.phi_or_time).T
    return new_fn


def sum_timescans(files, ddir, new_filename):
    datas = []
    for i in range(0, len(files)):
        ad = ARPES_DATA[files[i]]
        datas.append(ad.data)
    data_avg = np.sum(datas, axis=0)
    print(data_avg.shape)

    new_data = data_avg

    new_fn = os.path.join(ddir, new_filename)

    with h5py.File(
        new_fn, "w"
    ) as f:  # Note: 'w' creates a new empty file (or overwrites), use 'r+' to modify an existing file
        f["data"] = new_data.T
        axes_names = [
            "angles",
            "energies",
        ]  # Change these to match your axes labels
        axes = [ad.theta, ad.energy]
        for axis, name in zip(axes, axes_names):
            f[name] = np.atleast_2d(axis).T
        entry_group = f.require_group("entry1")
        entry_group["ScanValues"] = np.atleast_2d(ad.phi_or_time).T
    return new_fn

# Averaging Datasets

In [ ]:
# ddir = r"E:\atully\arpes_data\2023_June\C60\ARPES\TR"
# files = []
# # TR0
# # files = [f"TR0\TR_000_{i}.h5" for i in range(1, 4)]

# # TR1
# # files = [f"TR1\TR_000_{i}.h5" for i in range(1, 4)]
# # files.extend(f"TR1\TR_000_{i}.h5" for i in range(1, 2))

# # TR2
# # files = [f"TR2\TR_002_{i}.h5" for i in range(1, 4)]

# # TR0 & TR2
# # files = [f"TR0\TR_000_{i}.h5" for i in range(1, 4)]
# # files.extend(f"TR2\TR_002_{i}.h5" for i in range(1, 4))

# # TR4  # CT2
# # files = [f"TR4\TR_005_{i}.h5" for i in range(1, 15)]  # -1 to 1 ps

# # TR6  # CT2
# # files = [f"TR6\TR_006_{i}.h5" for i in range(1, 51)]  # -0.5 to 0.5 ps

# # TR7  # CT1
# # files = [
# #     f"TR7\TR_008_{i}.h5" for i in range(1, 83)
# # ]  # -1 to 1 ps in steps of 100 fs, then out to 30 ps with variable steps

# # TR9 & TR11  # CT2
# # files = [f"TR9\TR_009_{i}.h5" for i in range(1, 32)]
# # files.extend(f"TR11\TR_011_{i}.h5" for i in range(1, 53))

# # # TR12  # CT1
# # files = [
# #     f"TR12\TR_012_{i}.h5" for i in range(1, 45)  # check for cone
# # ]  # -1 to 1 ps in steps of 100 fs, then out to 30 ps with variable steps

# ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
# ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
# for file in tqdm(files):
#     data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
#     ARPES_DATA[file] = tr_functions.ArpesData(
#         data=data, theta=kx, phi_or_time=ky, energy=energy
#     )
#     ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

In [ ]:
# average_timescans(files, ddir, "TR0/TR0_Ali_avg.h5")
# average_timescans(files, ddir, "TR1/TR1_Ali_avg.h5")
# average_timescans(files, ddir, "TR2/TR2_Ali_avg.h5")
# average_timescans(files, ddir, "TR0&TR2_Ali_avg.h5")

# sum_timescans(files, ddir, "TR4_Ali_avg.h5")
# sum_timescans(files, ddir, "TR6_Ali_avg.h5")
# sum_timescans(files, ddir, "TR7_Ali_avg.h5")

# sum_timescans(files, ddir, "TR9&TR11_Ali_avg.h5")
# sum_timescans(files, ddir, "TR12_Ali_avg_best.h5")

# Load Data

In [ ]:
# # ## Feb Comparison ##

# ddir = r"E:\atully\arpes_data\2023_February\6eV\TR"
# files = []
# files = [
#     "TR4_avg_g_kw_filteredFFT_0.00int.h5"
# ]  # 2.6 eV center energy; -1 to 1 ps; homo400 offset

# ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
# ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
# for file in tqdm(files):
#     data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
#     ARPES_DATA[file] = tr_functions.ArpesData(
#         data=data, theta=kx, phi_or_time=ky, energy=energy
#     )
#     ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

In [ ]:
# ad = ARPES_DATA[files[0]]
# for k in ["energy", "theta", "phi_or_time"]:
#     print(f"{k}.shape = {getattr(ad, k).shape}")
# print(f"Data.shape = {ad.data.shape}")

# ad_feb_4 = ad

In [ ]:
ddir = r"E:\atully\arpes_data\2023_June\C60\ARPES\TR"
files = []

# files = [f"TR0/TR0_Ali_avg.h5"]
# files = [f"TR1/TR1_Ali_avg.h5"]
# files = [f"TR2/TR2_Ali_avg.h5"]
# files = [f"TR0&TR2_Ali_avg.h5"]

## High Statistics Scans -- at M to K##
files = [f"TR4_Ali_avg.h5"]
# files = [f"TR6_Ali_avg.h5"]  # -0.5 to 0.5 ps
# files = [
#     f"TR7_Ali_avg.h5"
# ]  # -1 to 1 ps in steps of 100 fs, then out to 30 ps with variable steps

## High Statistics Scans -- at G ##
# files = [f"TR9&TR11_Ali_avg.h5"]  # -0.5 to 0.5 ps
# files = [
#     f"TR12_Ali_avg_best.h5"
# ]  # -1 to 1 ps in steps of 100 fs, then out to 30 ps with variable steps

# ### February Comparison ###
# ddir = r"E:\atully\arpes_data\2023_February\6eV\TR"
# files = []
# # files = ["TR3_Ali_avg.h5"]
# # files = [
# #     "TR4_Ali_avg.h5"
# # ]  # 2.6 eV center energy; -1 to 1 ps, same number of steps as first 2 ps of TR3
# files = ["TR_001_1.h5"]

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

In [ ]:
# ad = ARPES_DATA[files[0]]
# for k in ["energy", "theta", "phi_or_time"]:
#     print(f"{k}.shape = {getattr(ad, k).shape}")
# print(f"Data.shape = {ad.data.shape}")

In [ ]:
# ad_feb_1 = ARPES_DATA[files[0]]
# ad_feb_4 = ARPES_DATA[files[0]]
# ad_7 = ARPES_DATA[files[0]]
# ad_12 = ARPES_DATA[files[0]]
# ad_4 = ARPES_DATA[files[0]]

In [ ]:
# ## High Statistics Scans -- at G ##
# files = []
# files = [f"TR9&TR11_Ali_avg.h5"]  # -0.5 to 0.5 ps
# # files = [
# #     f"TR12_Ali_avg.h5"
# # ]  # -1 to 1 ps in steps of 100 fs, then out to 30 ps with variable steps

# ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
# ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
# for file in tqdm(files):
#     data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
#     ARPES_DATA[file] = tr_functions.ArpesData(
#         data=data, theta=kx, phi_or_time=ky, energy=energy
#     )
#     ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

# ad_top = ARPES_DATA[files[0]]
# # ad_bottom = ARPES_DATA[files[0]]

# for k in ["energy", "theta", "phi_or_time"]:
#     print(f"{k}.shape = {getattr(ad, k).shape}")
# print(f"Data.shape = {ad.data.shape}")

# Analysis Setup

In [ ]:
## Zero Delay, HOMO, EF ##

time_zero = 34.8225  # Bi2Se3

time_zero_feb = 37.96  # February

EF_400 = 1.91  # in kinetic energy, slit 400
EF_700 = 1.94  # in kinetic energy, slit 700

homo = -2.05

homo_400 = homo + EF_400
homo_700 = homo + EF_700

In [ ]:
## Integrate over desired angular range ##

slice_dim = "x"
slice_val = 0
int_range = 20  # if this value is more that the integration range, my get_2D_slice function will just integrate over the max range.

xlim = None
ylim = None
x_bin = 1
y_bin = 1

In [ ]:
# ## Integrate over desired time range ##

# slice_dim = "z"
# slice_val = time_zero + tr_functions.ps_to_mm(0, time_zero)
# int_range = tr_functions.ps_to_mm(
#     0.5, time_zero
# )  # if this value is more that the integration range, my get_2D_slice function will just integrate over the max range.

# # xlim = (-20, 16.5)
# xlim = (-20, 15)
# ylim = None

# x_bin = 1
# y_bin = 1

In [ ]:
all_vals = []
for ad in [ad_feb_1, ad_feb_4, ad_12, ad_7, ad_4]:
    all_vals.append(
        tr_functions.slice_datacube(
            ad_dataclass=ad,
            slice_dim=slice_dim,
            slice_val=slice_val,
            int_range=int_range,
            xlim=xlim,
            ylim=(
                ad.energy[57],
                ad.energy[1007],
            ),  # get rid of zero padding on datasets
            x_bin=x_bin,
            y_bin=y_bin,
            norm_data=False,
            plot_data=False,
        )
    )

x_1, y_1, d_1 = all_vals[0]
x_4, y_4, d_4 = all_vals[1]
x_12, y_12, d_12 = all_vals[2]
x_7, y_7, d_7 = all_vals[3]
x_4_june, y_4_june, d_4_june = all_vals[4]

In [ ]:
## Adjust energy axis to be relative to HOMO and convert mm to ps ##

homo_zero = False
homo_zero = True

if homo_zero:
    # Feb TR4 and June #
    x4, y4, d4 = tr_functions.mm_to_ps(x_4, time_zero_feb), y_4 - homo_400, d_4
    x12, y12, d12 = tr_functions.mm_to_ps(x_12, time_zero), y_12 - homo_400, d_12
    x7, y7, d7 = tr_functions.mm_to_ps(x_7, time_zero), y_7 - homo_400, d_7
    x4_june, y4_june, d4_june = (
        tr_functions.mm_to_ps(x_4_june, time_zero),
        y_4_june - homo_400,
        d_4_june,
    )

    # Feb TR1 #
    x1, y1, d1 = tr_functions.mm_to_ps(x_1, time_zero_feb), y_1 - homo_700, d_1

# Analysis

In [ ]:
title = f"C<sub>60</sub> Pump Effect: {files[0]}"
# title = f"CT<sub>2</sub>"
title = f"C<sub>60</sub> Pump Effect"

yaxis_title = "E - E<sub>HOMO</sub> (eV)"
xaxis_title = "Delay (ps)"
# xaxis_title=f"k<sub>x</sub> [{angstrom}<sup>-1</sup>]"

In [ ]:
## Plot Data ##
# x_plot, y_plot, d_plot = x1, y1, d1
x_plot, y_plot, d_plot = x4, y4, d4
# x_plot, y_plot, d_plot = x12, y12, d12
# x_plot, y_plot, d_plot = x7, y7, d7
x_plot, y_plot, d_plot = x4_june, y4_june, d4_june

# # FFT angular dispersion data
# d_2d = filter_functions.fft2d_mask(d_2d, plot=False)

fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    height=600,
    width=800,
    dtick_y=0.1,
)

fig.add_trace(
    go.Heatmap(
        x=x_plot,
        y=y_plot,
        z=analysis_functions.norm_data(d_plot),
        coloraxis="coloraxis",
    )
)

# fig.update_coloraxes(colorscale="greys", showscale=False)
# fig.update_coloraxes(cmin=0.0, cmax=0.8)

fig.show()

In [ ]:
## Integrated over energy ##

xlim = None
# ylim = None
ylim = (1.95, 2.3)
ylim = (2.05, 2.3)
ylim = (2.0, 2.3)

## Plot data
if ylim is None:
    title = f"Average over all Energy"
else:
    title = f"Average over {ylim} eV"
fig = tr_functions.thesis_fig(
    # title=title,
    title=f"CT<sub>1</sub> Lifetime Comparison",
    xaxis_title="Delay (ps)",
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

## Get slice(s)

ads = []
ads = [ad_feb_1, ad_12, ad_7]

names = ["G (500 fs)", "G - K (250 fs)", "M - K (250 fs)"]

for i, ad in enumerate(ads):
    color = colors[i % len(colors)]

    x_2d, y_2d, d_2d = tr_functions.slice_datacube(
        ad_dataclass=ad,
        slice_dim=slice_dim,
        slice_val=slice_val,
        int_range=int_range,
        xlim=None,
        # ylim=ylim,
        ylim=(
            ad.energy[57],
            ad.energy[1007],
        ),  # get rid of zero padding on datasets
        x_bin=1,
        y_bin=1,
        norm_data=False,
        plot_data=False,
    )

    if i == 0:
        y_2d = y_2d - homo_700
    else:
        y_2d = y_2d - homo_400

    # Convert mm to ps
    if i == 0:
        x_2d = tr_functions.mm_to_ps(x_2d, time_zero_feb)
    else:
        x_2d = tr_functions.mm_to_ps(x_2d, time_zero)

    x_1d, row = tr_functions.get_1d_y_slice(
        x=x_2d,
        y=y_2d,
        data=d_2d,
        xlims=None,
        y_range=ylim,
    )

    data = analysis_functions.norm_data(row)
    fig.add_trace(
        go.Scatter(
            x=x_1d,
            y=data,
            name=f"{names[i]}",
            mode="lines+markers",
            # mode="markers",
            line=dict(color=color),
        )
    )

    fig.update_layout(
        legend=dict(
            font=dict(size=14),
        )
    )
    # fig.add_trace(go.Scatter(x=x_1d, y=row - np.mean(row[:4]), name=f"{names[i]}"))
    # fig.add_trace(go.Scatter(x=x_1d, y=row, name=f"{names[i]}"))

    ## Fit ##
    offset_type = "constant"
    x = x_1d[np.where(x_1d > 0)]
    data_fit = data[np.where(x_1d > 0)]

    c = np.mean(data)
    b = (data[-1] - data[0]) / (x[-1] - x[0])
    a = 0

    offset = fitting_functions.offset_model(offset_type, a, b, c)
    exp = lm.models.ExponentialModel()

    full_model = exp + offset

    params = full_model.make_params()

    params["decay"].value = 10
    params["amplitude"].value = 1

    fit = full_model.fit(data_fit, x=x, params=params)
    print(f"Decay: {fit.params['decay'].value}")
    # fit.plot()

    # fig.add_trace(
    #     go.Scatter(
    #         x=x,
    #         # y=analysis_functions.norm_data(row),
    #         y=fit.eval(x=x),
    #         # name=f"{names[i]}",
    #         line=dict(color=color),
    #     )
    # )

    fig.add_annotation(
        # x=fit.params[f"{model_name}center"].value,
        # y=fit.eval(x=fit.params[f"{model_name}center"].value),
        # xref="paper",
        # yref="paper",
        x=20,
        y=0.8 - i * 0.1,
        # align="right",
        showarrow=False,
        text=f'Decay: {fit.params[f"decay"].value:.2f} +/- {fit.params[f"decay"].stderr:.2f} ps',
        font=dict(size=14),
        # ax=300,
        # ay=i * 50,
        bgcolor="white",
        # opacity=1,
        bordercolor=color,
        borderwidth=2,
        # borderpad=4,
    )

fig.update_xaxes(range=(-1, 30))
# fig.update_xaxes(range=(-1, 20))
fig.show()

In [ ]:
x_1d[np.where(x_1d >= 0)]

In [ ]:
import lmfit as lm

offset_type = "constant"
x = x_1d[np.where(x_1d > 0)]
data = row[np.where(x_1d > 0)]

c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

# gaussian = fitting_functions.make_gaussian(
#     num="A",
#     amplitude=1,
#     center=0,
#     sigma=0.05,
#     include_exp_decay=True,
#     gamma=50,
#     # lock_sigma=True,  # sigma should be based on fwhm of BiSe, but the pump profile changes
# )

exp = lm.models.ExponentialModel()

full_model = exp + offset

params = full_model.make_params()

params["decay"].value = 10
params["amplitude"].value = 1


fit = full_model.fit(data, x=x, params=params)

fit.plot()

In [ ]:
fit

In [ ]:
## Integrated over energy ##

xlim = None
ylim = None

## Plot data
if ylim is None:
    title = f"Average over all Energy"
else:
    title = f"Average over {ylim} eV"
fig = tr_functions.thesis_fig(
    # title=title,
    title="CT<sub>2</sub> Lifetime Comparison",
    xaxis_title="Delay (ps)",
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

## Get slice(s)

x_1d, row = tr_functions.get_1d_y_slice(
    x=x4,
    y=y4,
    data=d4,
    xlims=None,
    y_range=ylim,
)

x_1d_june, row_june = tr_functions.get_1d_y_slice(
    x=x4_june,
    y=y4_june,
    data=d4_june,
    xlims=None,
    y_range=ylim,
)


fig.add_trace(
    go.Scatter(
        x=x_1d,
        y=analysis_functions.norm_data(row - np.mean(row[:4])),
        name=f"500 fs",
        mode="lines+markers",
    )
)
fig.add_trace(
    go.Scatter(
        x=x_1d_june,
        y=analysis_functions.norm_data(row_june - np.mean(row_june[:4])),
        name=f"250 fs",
        mode="lines+markers",
    )
)
# fig.add_trace(go.Scatter(x=x_1d, y=row - np.mean(row[:4]), name=f"{names[i]}"))
# fig.add_trace(go.Scatter(x=x_1d, y=row, name=f"{names[i]}"))

fig.update_xaxes(range=(-1, 1))
fig.show()

In [ ]:
import lmfit as lm

offset_type = "constant"
x = x_1d
data = row

c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

gaussian = fitting_functions.make_gaussian(
    num="A",
    amplitude=1,
    center=0,
    sigma=0.05,
    include_exp_decay=True,
    gamma=50,
    # lock_sigma=True,  # sigma should be based on fwhm of BiSe, but the pump profile changes
)

# gaussian = lm.models.ExponentialGaussianModel()

full_model = gaussian + offset

params = full_model.make_params()

# params["center"].value = 0
# params["amplitude"].value = 1
# params["sigma"].value = 0.1
# params["gamma"].value = 100


fit = full_model.fit(data, x=x, params=params)

fit.plot()

In [ ]:
fit

In [ ]:
import lmfit as lm

offset_type = "constant"

x = x_1d_june
data = row_june

c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

# gaussian = fitting_functions.make_gaussian(
#     num="A",
#     amplitude=1,
#     center=0,
#     sigma=0.05,
#     include_exp_decay=True,
#     gamma=19,
#     # lock_sigma=True,  # sigma should be based on fwhm of BiSe, but the pump profile changes
# )

gaussian = lm.models.ExponentialGaussianModel()

full_model = gaussian + offset

params = full_model.make_params()

params["center"].value = 0
params["amplitude"].value = 1
params["sigma"].value = 0.05
params["gamma"].value = 20


fit = full_model.fit(row, x=x_1d, params=params)

fit2.plot()

In [ ]:
## Integrated over energy ##

# x_plot, y_plot, z_plot = x_2d, y_2d, d_2d

xlim = None
# ylim = None
ylim = (1.95, 2.3)
ylim = (2.05, 2.3)
ylim = (2.0, 2.3)

# ylim = (1.95, 2.41)  # looking for pump effect multiple of background
# ylim = (1.95, 2.15)  # looking for peak of CT1

## Plot data
title = f"Average over various energies"
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title="Delay (ps)",
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

## Get slice(s)

ylims = []
ylims = [(2.0, 2.3), (1.7, 2.3)]

# names = ["Feb: G", "June: G - K", "June: M - K"]

for i, ylim in enumerate(ylims):
    color = colors[i % len(colors) + 1]

    #     x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    #         ad_dataclass=ad_feb_1,
    #         slice_dim=slice_dim,
    #         slice_val=slice_val,
    #         int_range=int_range,
    #         xlim=None,
    #         # ylim=ylim,
    #         ylim=(
    #             ad.energy[57],
    #             ad.energy[1007],
    #         ),  # get rid of zero padding on datasets
    #         x_bin=1,
    #         y_bin=1,
    #         norm_data=False,
    #         plot_data=False,
    #     )

    #     y_2d = y_2d - homo_700

    #     # Convert mm to ps
    #     x_2d = tr_functions.mm_to_ps(x_2d, time_zero_feb)

    x_1d, row = tr_functions.get_1d_y_slice(
        x=x1,
        y=y1,
        data=d1,
        xlims=None,
        y_range=ylim,
    )

    # fig.add_trace(
    #     go.Scatter(x=x_1d, y=analysis_functions.norm_data(row), name=f"{ylim}")
    # )
    fig.add_trace(go.Scatter(x=x_1d, y=row - np.mean(row[:4]), name=f"{ylim}"))
    # fig.add_trace(
    #     go.Scatter(
    #         x=x_1d,
    #         y=analysis_functions.norm_data(row - np.mean(row[:4])),
    #         name=f"{ylim}",
    #     )
    # )
    # fig.add_trace(go.Scatter(x=x_1d, y=row, name=f"{ylim}"))

x_1d_2, row_2 = tr_functions.get_1d_y_slice(
    x=x4,
    y=y4,
    data=d4,
    xlims=None,
    y_range=(2.4, 3.0),
)

# fig.add_trace(
#     go.Scatter(x=x_1d_2, y=analysis_functions.norm_data(row_2), name=f"(2.4, 3.0)")
# )
fig.add_trace(go.Scatter(x=x_1d_2, y=row_2 - np.mean(row_2[:4]), name=f"(2.4, 3.0)"))
# fig.add_trace(
#     go.Scatter(
#         x=x_1d_2,
#         y=analysis_functions.norm_data(row_2 - np.mean(row_2[:4])),
#         name=f"(2.4, 3.0)",
#     )
# )
# fig.add_trace(go.Scatter(x=x_1d_2, y=row_2, name=f"(2.4, 3.0)"))

# fig.update_xaxes(range=(-1, 30))
fig.show()

In [ ]:
T = 10.6  # measurement temp
k_B = 8.617333e-5  # eV/K

title = "Waterfall Plot of G - K"
# x_plot, y_plot, z_plot = analysis_functions.limit_dataset(
#     x12, y12, d12, xlim=(np.min(x12), np.mean(x12)), ylim=(2, np.max(y12))
# )

x_plot, y_plot, z_plot = x12, y12, d12

z_plot = analysis_functions.norm_data(z_plot)

# title = "Waterfall Plot of M - K"
# x_plot, y_plot, z_plot = x7, y7, analysis_functions.norm_data(d7)

spacing = 0.2

if np.min(x_plot) > 0:
    x_edc = tr_functions.mm_to_ps(x_plot, time_zero)
else:
    x_edc = x_plot

slice_window = 10
steps = 20
new_axes = []
new_data = []
fits = []
fit_components = []
ylim = None
xlim = (np.min(x_edc), np.max(x_edc))
title = f"{title} ({slice_window} ps window)"

print(f"Minimum window size: {np.max(np.diff(x_edc))}")

fig = tr_functions.thesis_fig(
    title=title,
    yaxis_title=yaxis_title,
    xaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=400,
)

for i, xval in enumerate(np.linspace(xlim[0], xlim[1], steps)):
    x_window = (xval - slice_window / 2, xval + slice_window / 2)
    # print(x_window)
    # if x_window[1] - x_window[0] <= np.max(
    #     np.diff(x_edc)
    # ):  # gives resolution = minimum window size
    #     raise ValueError(f"{x_window} is <= min window size ({np.max(np.diff(x_edc))})")
    y_1d, col = tr_functions.get_1d_x_slice(
        x=x_edc,
        y=y_plot,
        data=z_plot,
        ylims=ylim,
        x_range=x_window,
    )

    if len(col) == 1:
        continue

    # if i == 10:
    #     print(y_1d, col)

    fig.add_trace(
        go.Scatter(
            y=y_1d,
            x=i * spacing + col,
            # x=xval + col * 20,
            name=f"{np.round(x_window[0], 1), np.round(x_window[1], 1)} ps",
            # line=dict(color=colors[i]),
            # line=dict(color=colors_seq[i + 1]),
            line=dict(color=colors[0]),
        )
    )
    new_axes.append(y_1d)
    new_data.append(col)

    x, data = y_1d, col
    offset_type = "constant"

    ## FD
    def fermi_dirac(x, center, theta, amp):
        arg = (x - center) / (2 * theta)  # x=E, center=mu, theta = k_B * T
        return -amp / 2 * np.tanh(arg)

    ## Offset
    c = np.mean(data)
    b = (data[-1] - data[0]) / (x[-1] - x[0])
    a = 0

    offset = fitting_functions.offset_model(offset_type, a, b, c)

    # Fit for 2 peaks
    # gauss1 = fitting_functions.make_gaussian(
    #     num="A_", amplitude=1, center=1.8, sigma=0.1
    # )  # 1.86
    gauss2 = fitting_functions.make_gaussian(
        num="B_", amplitude=1, center=2.2, sigma=0.1
    )  # 2.1
    gauss3 = fitting_functions.make_gaussian(
        num="C_", amplitude=1, center=2.55, sigma=0.1
    )  # 2.1

    ## Full model
    full_model = lm.models.Model(fermi_dirac) + gauss2 + gauss3 + offset

    params = full_model.make_params()

    if offset_type == "constant":
        # params = linear_params.copy()
        params = params
        params["b"].value = 0
        params["b"].vary = False

    params[
        "center"
    ].value = (
        2.0325  # TR12, determined by fitting from (0, 100) ps with a constant offset
    )
    # params["center"].vary = False
    # # params["c"].value = 0  # determined by fitting from (0, 100) ps with a constant offset
    # # params["c"].vary = False
    params["theta"].value = k_B * (10.6)
    params["theta"].min = 0
    params["amp"].value = 1
    params["amp"].min = 0

    # params["iA__center"].max = 1.95
    # params["iA__center"].min = 1.6
    # params["iA__sigma"].min = tr_functions.fwhm_to_sig(0.1)
    # params["iA__sigma"].vary = False
    params["iB__center"].min = 2.0
    params["iB__sigma"].value = tr_functions.fwhm_to_sig(0.2)
    params["iB__sigma"].max = tr_functions.fwhm_to_sig(0.4)
    params["iC__center"].min = 2.4
    params["iC__sigma"].value = tr_functions.fwhm_to_sig(0.1)
    # params["iC__sigma"].max = tr_functions.fwhm_to_sig(0.2)
    params["iC__sigma"].max = tr_functions.fwhm_to_sig(0.4)

    fit = full_model.fit(data, x=x, params=params)

    # if fit.params["amp"].stderr:
    fits.append(fit)
    fig.add_trace(
        go.Scatter(
            x=i * spacing + fit.eval(x=y_1d),
            y=y_1d,
            name="fit",
            line=dict(color="red"),
            opacity=0.5,
        )
    )

    components = fit.eval_components(x=y_1d)
    fit_components.append(components)

#     if plot_fit:
#         fit.plot()

#     if offset_type == "linear":
#         linear_params = fit.params.copy()

#     ## collect fixed center values
#     if collect_centers:
#         centers_fixed = []
#         components = fit.eval_components(x=y_1d)
#         for model_name, model_value in list(components.items())[1:8]:
#             centers_fixed.append(fit.params[f"{model_name}center"].value.astype("float32"))


fig.show()

In [ ]:
xs = np.linspace(xlim[0], xlim[1], steps)
colors = pc.qualitative.D3

for param_name in ["amplitude", "height", "fwhm", "center"]:
    # Fig for each fitting parameter
    fig = tr_functions.default_fig()
    fig.update_layout(
        title=param_name,
        xaxis_title="delay (ps)",
        yaxis_title=param_name,
        font=dict(size=20),
    )

    # For each peak individually (one trace per peak)
    for i, letter in enumerate("BC"):
        color = colors[i % len(colors) + 1]
        ys = []
        y_errs = []
        y_inits = []
        for fit in fits:
            par = fit.params[f"i{letter}__{param_name}"]
            ys.append(par.value)
            y_errs.append(par.stderr)
            y_inits.append(par.init_value)

        fig.add_trace(
            go.Scatter(
                x=xs,
                y=ys,
                error_y=dict(array=y_errs),
                name="value",
                line=dict(color=color),
                legendgroup=letter,
                legendgrouptitle_text=letter,
                mode="markers+lines",
            )
        )
    fig.show()

In [ ]:
## EDCs with different time integration ##

title = f"{files[0]}"
spacing = 0

x_plot, y_plot, z_plot = x_2d, y_2d, d_2d  # regular

if np.min(x_plot) > 0:
    x_edc = tr_functions.mm_to_ps(x_plot, time_zero)
else:
    x_edc = x_plot


## Normalize?
normalize = False
# normalize = True

## Logplot?
logplot = False
# logplot = True

if logplot:
    title = f"{title}<br>(logplot)"
else:
    title = f"{title}"


## Set up integration limits
# xlim1 = (-2, -1)
# xlim2 = (-0.5, 0.5)
# xlim3 = (1, 2)

xlim1 = (-0.5, 0)
xlim2 = (-0.25, 0.25)
xlim3 = (0, 0.5)

# ## check for pre-timezero intensity
# xlim1 = (-2, -1.5)
# xlim2 = (-1.5, -1)
# xlim3 = (-1, -0.5)
# ylim = (1.95, 2.45)

xlims = [xlim1, xlim2, xlim3]

# xlim = None
ylim = None


# Get and plot data

fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=yaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
)

for i, xlim in enumerate(xlims):
    color = colors[i % len(colors)]

    y_1d, col = tr_functions.get_1d_x_slice(
        x=x_edc,
        y=y_plot,
        data=z_plot,
        ylims=ylim,
        x_range=xlim,
    )

    if normalize:
        col = analysis_functions.norm_data(col)

    if logplot:
        col = np.log(col)

    # Plot Data
    fig.add_trace(
        go.Scatter(
            x=y_1d,
            y=i * spacing + col,
            name=f"{np.round(xlim[0], 2), np.round(xlim[1], 2)} ps",
            line=dict(color=color),
        )
    )

fig.show()

# Difference Map

In [ ]:
## Difference Map ##
title = f"Difference Map of {files[0]}"
x, y, d = x_2d, y_2d, d_2d

d_diff = d - np.mean(d[:, 0:4], axis=1)[:, None]

## Plot Data
fig = tr_functions.thesis_fig(
    title=f"{title}",
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    gridlines=False,
    # height=400,
    height=600,
    width=800,
    dtick_y=0.1,
)

# x_plot = tr_functions.mm_to_ps(x, time_zero)
x_plot = x
y_plot = y

fig.add_trace(go.Heatmap(x=x_plot, y=y_plot, z=d_diff, coloraxis="coloraxis"))
# for h in [1.63, 1.8, 1.98]:
#     fig.add_hline(y=h, line=dict(color="black", width=1, dash="dash"))
fig.update_coloraxes(colorscale="RdBu", cmid=0, showscale=True)

In [ ]:
## EDCs with different time integration ##

title = f"{files[0]}"
spacing = 0

x_plot, y_plot, z_plot = x_2d, y_2d, d_2d  # regular
# x_plot, y_plot, z_plot = x_2d, y_2d, d_diff  # diffmap

if np.min(x_plot) > 0:
    x_edc = tr_functions.mm_to_ps(x_plot, time_zero)
else:
    x_edc = x_plot


## Normalize?
normalize = False
# normalize = True

## Logplot?
logplot = False
# logplot = True

if logplot:
    title = f"{title}<br>(logplot)"
else:
    title = f"{title}"


## Set up integration limits
xlim1 = (-1, 0)
xlim2 = (0, 1)
xlim3 = (1, 5)
# xlim4 = (0, 10)

xlims = [xlim1, xlim2, xlim3]

# xlim = None
ylim = None


# Get and plot data

fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=yaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
)

for i, xlim in enumerate(xlims):
    color = colors[i % len(colors)]

    y_1d, col = tr_functions.get_1d_x_slice(
        x=x_edc,
        y=y_plot,
        data=z_plot,
        ylims=ylim,
        x_range=xlim,
    )

    if normalize:
        col = analysis_functions.norm_data(col)

    if logplot:
        col = np.log(col)

    # Plot Data
    fig.add_trace(
        go.Scatter(
            x=y_1d,
            y=i * spacing + col,
            name=f"{np.round(xlim[0], 2), np.round(xlim[1], 2)} ps",
            line=dict(color=color),
        )
    )

fig.show()

# Fitting Time Res

In [ ]:
## Integrated over energy ##

xlim = None
ylim = None
# ylim = (3.3, 3.545)

## Get slice
x_1d, row = tr_functions.get_1d_y_slice(
    x=x_2d,
    y=y_2d,
    data=d_2d,
    xlims=xlim,
    y_range=ylim,
)

## Plot data
if ylim is None:
    ylim = "all"
fig = tr_functions.thesis_fig(
    title=f"Average over {ylim} eV",
    xaxis_title="Delay (ps)",
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

fig.add_trace(go.Scatter(x=x_1d, y=row, name="data"))

## For After Fit
fig.add_trace(go.Scatter(x=x, y=fit.eval(x=x_1d), name="fit"))

components = fit.eval_components(x=x_1d)
for model_name, model_value in list(components.items())[0:1]:
    fig.add_annotation(
        # x=fit.params[f"center"].value,
        # y=fit.eval(x=fit.params[f"{model_name}center"].value),
        xref="x domain",
        yref="y domain",
        x=0.01,
        y=0.99,
        showarrow=False,
        # text=f'Peak center: {fit.params[f"center"].value:.2f} +/- {fit.params[f"center"].stderr:.4f} mm<br>FWHM: {tr_functions.mm_to_ps(tr_functions.sig_to_fwhm(fit.params["sigma"].value)):.3f} +/- {tr_functions.mm_to_ps(tr_functions.sig_to_fwhm(fit.params[f"sigma"].stderr)):.4f} ps',
        text=f'Peak center:<br>{fit.params[f"center"].value:.2f} +/- {fit.params[f"center"].stderr:.4f} ps<br><br>FWHM:<br>{tr_functions.sig_to_fwhm(fit.params["sigma"].value):.3f} +/- {tr_functions.sig_to_fwhm(fit.params[f"sigma"].stderr):.4f} ps',
        # text=f'Peak:<br>{np.round(x_1d[np.where(fit.eval(x=x_1d) == np.max(fit.eval(x=x_1d)))][0], 2)} ps<br><br>FWHM:<br>{tr_functions.sig_to_fwhm(fit.params["sigma"].value):.3f} +/- {tr_functions.sig_to_fwhm(fit.params[f"sigma"].stderr):.4f} ps',
        font=dict(size=18),
    )

fig.show()

print(
    f'center: {np.round(fit.params["center"].value, 2)} +/- {fit.params[f"center"].stderr:.4f} ps'
)
print(
    f'decay: {np.round(1 / fit.params["gamma"].value, 3) * 1000} +/- {1 / fit.params[f"gamma"].stderr:.4f} fs'
)
# print(f"peak: {x_1d[np.where(fit.eval(x=x_1d) == np.max(fit.eval(x=x_1d)))][0]}")
print(
    f"peak: {np.round(x_1d[np.where(fit.eval(x=x_1d) == np.max(fit.eval(x=x_1d)))][0], 2)} ps"
)
print(
    f'FWHM: {tr_functions.sig_to_fwhm(fit.params["sigma"].value):.3f} +/- {tr_functions.sig_to_fwhm(fit.params[f"sigma"].stderr):.4f} ps'
)

In [ ]:
## Plot Residuals
fig = tr_functions.thesis_fig(
    title=f"{title}",
    xaxis_title=xaxis_title,
    yaxis_title="Residuals",
    equiv_axes=False,
    gridlines=False,
    height=400,
    width=900,
)

fig.add_trace(go.Scatter(x=x_1d, y=row - fit.eval(x=x_1d), name="fit"))
fig.add_hline(y=0, line=dict(dash="dash"))

In [ ]:
## Attempt to manually build convolved model: This is the built-in model ##

import lmfit as lm

offset_type = "constant"
x = x_1d
data = row

c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

# gaussian = fitting_functions.make_gaussian(
#     num="A",
#     amplitude=1,
#     center=0,
#     sigma=0.05,
#     include_exp_decay=True,
#     gamma=19,
#     # lock_sigma=True,  # sigma should be based on fwhm of BiSe, but the pump profile changes
# )

gaussian = lm.models.ExponentialGaussianModel()

full_model = gaussian + offset

params = full_model.make_params()

params["center"].value = 0
params["amplitude"].value = 1
params["sigma"].value = 0.05
params["gamma"].value = 20


fit = full_model.fit(row, x=x_1d, params=params)

fit.plot()

In [ ]:
fit

In [ ]:
tr_functions.sig_to_fwhm(fit.params["sigma"].value)